
## Google analytics customer revenue predictions
### By Ahmed Eweed  

### You can find me on github at [github.com/AhmedEweed](https://github.com/AhmedEweed)

    I re approached the dataset as I exploed it in many ways
    and this version is the updated version of my exploration

In [17]:
#import needed libraries
import pandas as pd
from pandas.io.json import json_normalize

#let's group JSON columns to work on them together
JSON_cols = ['device', 'geoNetwork', 'totals', 'trafficSource']

#read data 
# I solved the mixed datatype problem in column 3 'fullVisitorId' by defining it to be string 
# that should stop the warning message.

# also I lowered down the number of rows displyed to 10k
# as I faced memory issues handling the whole 900k entries!

df = pd.read_csv("../train.csv", 
                     converters={column: json.loads for column in JSON_cols}, 
                     dtype={'fullVisitorId': 'str'}, nrows = 10000)

# we will make a copy of the dataset to work on and clean 
df_copy = df.copy()

# flatten json columns into table values
for column in JSON_cols:
    column_as_df = json_normalize(df_copy[column])
    column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
    df_copy = df_copy.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

In [18]:
#look at dataset dimensions

df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 54 columns):
channelGrouping                                      10000 non-null object
date                                                 10000 non-null int64
fullVisitorId                                        10000 non-null object
sessionId                                            10000 non-null object
socialEngagementType                                 10000 non-null object
visitId                                              10000 non-null int64
visitNumber                                          10000 non-null int64
visitStartTime                                       10000 non-null int64
device.browser                                       10000 non-null object
device.browserSize                                   10000 non-null object
device.browserVersion                                10000 non-null object
device.deviceCategory                                10000 non-null obj

    the dataset has 10k rows and 54 columns
    8 of those columns are strings and 4 are numbers or so says the dataset
    we have 4 columns that contain json data ('device', 'geoNetwork', 'totals', 'trafficSource' )
    let's have a deeper look at each column's data!

In [24]:
df_copy['totals.transactionRevenue'].value_counts()

30390000      2
25000000      2
44790000      2
10990000      2
13590000      2
11960000      1
63840000      1
105120000     1
1077000000    1
24080000      1
21020000      1
305470000     1
19990000      1
37960000      1
43710000      1
8980000       1
31930000      1
298760000     1
34480000      1
244040000     1
95970000      1
6990000       1
80760000      1
31980000      1
8380000       1
57570000      1
16990000      1
574150000     1
335310000     1
7990000       1
             ..
90850000      1
546320000     1
74590000      1
35480000      1
259370000     1
173880000     1
13290000      1
15190000      1
35080000      1
1990000       1
31490000      1
139650000     1
79990000      1
111160000     1
264790000     1
339030000     1
33670000      1
9980000       1
17590000      1
306670000     1
300350000     1
109150000     1
113520000     1
16970000      1
117340000     1
7480000       1
593910000     1
10590000      1
33980000      1
64700000      1
Name: totals.transaction

In [25]:
# look at 'channelGrouping' values
df_copy['channelGrouping'].value_counts()

Organic Search    4730
Social            1851
Direct            1645
Referral          1190
Paid Search        314
Affiliates         201
Display             69
Name: channelGrouping, dtype: int64

In [27]:
# look at 'date' column
# date is in Y/M/D format Remember!
df_copy['date'].head()

0    20160902
1    20160902
2    20160902
3    20160902
4    20160902
Name: date, dtype: int64

    this is definitely wrong! as date is represented by integers

In [28]:
# now convert the str to datetime on the format Y-M-D
df_copy['date'] = pd.to_datetime(df_copy['date'], format = '%Y%m%d')

In [29]:
# make sure the conversion is done 
df_copy['date'].head()

0   2016-09-02
1   2016-09-02
2   2016-09-02
3   2016-09-02
4   2016-09-02
Name: date, dtype: datetime64[ns]